In [3]:
# default_exp db.mysql

%reload_ext autoreload
%autoreload 2

# pyMySQL
https://github.com/PyMySQL/PyMySQL

https://pymysql.readthedocs.io/en/latest/

## install

In [3]:
# !pip install pymysql -U

^C
ERROR: Operation cancelled by user


In [1]:
!pip freeze | grep PyMySQL

PyMySQL==0.9.3


## 原生 PyMySQL

In [1]:
import pymysql

In [15]:
conn = pymysql.connect(user='root', passwd='123456', db='dics', host='127.0.0.1', use_unicode=True, charset='utf8')

In [11]:

conn = pymysql.connect(user='admin', passwd='nTs3mW9&qPzA', db='hcp_cs_prod', host='hcp-test-mysql.internal.leapstack.cn', use_unicode=True, charset='utf8')


In [12]:
cursor = conn.cursor()

In [13]:
sqltemplate = """
        show tables
        """
cursor.execute(sqltemplate)

86

In [7]:
cursor.fetchall()

(('ICD10_data',),
 ('api_hospital_transform',),
 ('backup_duty_data',),
 ('backup_insurance_type_data',),
 ('backup_insurance_type_to_duty_data',),
 ('backup_product_data',),
 ('backup_product_to_insurance_type_data',),
 ('backup_product_value_data',),
 ('backup_tmp_product_info',),
 ('bank_code',),
 ('bank_meta',),
 ('claim_apply_batch',),
 ('claim_apply_record',),
 ('claim_data',),
 ('claim_data_feedback',),
 ('claim_data_history',),
 ('claim_hcp_result',),
 ('claim_history',),
 ('claim_id_msg',),
 ('claim_insurance_entity',),
 ('claim_mark',),
 ('claim_meta',),
 ('claim_remark',),
 ('claim_risk_request',),
 ('claim_special_duty',),
 ('claim_status',),
 ('claim_status_logger',),
 ('claim_tentative_calculation',),
 ('claim_tentative_calculation_detail',),
 ('claim_third_suspend',),
 ('claim_trace',),
 ('custom_report',),
 ('dict_coverter_data',),
 ('dictionary_data',),
 ('duty_data',),
 ('duty_info',),
 ('effective_time_rule',),
 ('effective_worktime_config',),
 ('event_data',),
 ('ev

In [14]:
sqltemplate = """
        SELECT * from person_info_aes1 limit 1;
        """

cursor.execute(sqltemplate)

cursor.fetchmany(5)

((1,
  '07af91f57ac71ca1884c479919703ebb',
  2,
  4,
  'f9f533e1e8e7d27d9a2e6bebc8cb6ff0',
  None,
  None,
  None,
  '',
  'd41d8cd98f00b204e9800998ecf8427e',
  None,
  '1fe372b4dd21164c586ca1b1e171b41f',
  'd41d8cd98f00b204e9800998ecf8427e',
  '',
  '',
  3393,
  3277,
  1402,
  1447,
  '2aab1d17f47b4689173115c8fa97d538',
  3376,
  3378,
  1,
  -1,
  -1,
  datetime.datetime(2018, 7, 20, 21, 41, 41, 442000),
  datetime.datetime(2018, 7, 20, 21, 41, 41, 442000),
  None,
  None,
  None),)

In [24]:
for i in range(3, 10):
    sqltemplate = """
            INSERT INTO disease_dic SET code = "{}", name = "{}"
            """
    print(sqltemplate.format(f'code{i}', f'name{i}'))
    cursor.execute(sqltemplate.format(f'code{i}', f'name{i}'))


            INSERT INTO disease_dic SET code = "code3", name = "name3"
            

            INSERT INTO disease_dic SET code = "code4", name = "name4"
            

            INSERT INTO disease_dic SET code = "code5", name = "name5"
            

            INSERT INTO disease_dic SET code = "code6", name = "name6"
            

            INSERT INTO disease_dic SET code = "code7", name = "name7"
            

            INSERT INTO disease_dic SET code = "code8", name = "name8"
            

            INSERT INTO disease_dic SET code = "code9", name = "name9"
            


### 注意：执行以下查询操作，会查询到刚刚插入的数据，但是实际的物理数据库中的数据并没有变化

In [25]:
sqltemplate = """
        SELECT * from disease_dic
        """

cursor.execute(sqltemplate)

cursor.fetchall()

(('code1', 'name1'),
 ('code2', 'name2'),
 ('code3', 'name3'),
 ('code4', 'name4'),
 ('code5', 'name5'),
 ('code6', 'name6'),
 ('code7', 'name7'),
 ('code8', 'name8'),
 ('code9', 'name9'))

### conn.commit()
Commit changes to stable storage.

注意：当发生数据库插入操作时，execute后数据表的数据只会在内存中记录，而不会提交到物理硬盘中，所以物理硬盘的数据是不会发生改变的。只有执行commit操作，数据库中的数据才会变化

### best practice:
可以在数据连续发生多次的insert, update, delete操作完成后，执行一次commit，把改变提交到物理数据库中，避免数据发生丢失！

In [26]:
conn.commit()

## PyMysqlWrapper

In [4]:
# export
import pymysql


class PyMysqlWrapper:
    """
    常用是sql场景：
    查询不重复的记录：
        select distinct name from user;
        select id,name from user group by name;
    """
    def __init__(self, user='root', passwd='123456', db='test', host='127.0.0.1', use_unicode=True, charset='utf8'):
        """
        创建连接、创建游标
        :param user:
        :param passwd:
        :param db:
        :param host:
        :param use_unicode:
        :param charset:
        :return:
        """
        self._conn = pymysql.connect(user=user, passwd=passwd, db=db, host=host, use_unicode=use_unicode, charset=charset)
        self._cursor = self._conn.cursor()

    def getCursor(self):
        return self._cursor

    def execute(self, sql, args=None):
        """
        Execute a query 并且提交,返回查询结果条数
        :return:
        """
        n = self._cursor.execute(sql, args)
        return n

    def excuteMany(self, sql, args):
        """

        """
        n = self._cursor.executemany(sql, args)
        return n

    def commit(self):
        """
        """
        self._conn.commit()

    def queryAndFetchAll(self, sql, return_df=True, cols=None):
        """
        针对查询的便利方法
        :param sql:
        :param return_df:  是否返回df格式
        :param cols:  如果以df格式返回时的col_name
        :return: 查询结果
            (('code1', 'name1'),
            ('code2', 'name2'),
            ……)
        """
        self._cursor.execute(sql)
        r = self._cursor.fetchall()
        if return_df:
            import pandas as pd
            return pd.DataFrame(list(r), columns=cols)
        return r

    def close(self):
        self._cursor.close()
        self._conn.close()

# test

In [9]:
mysql = PyMysqlWrapper(user='ro_user', passwd='boishYo9Voo',
                db='claim_registration_center', host='hcp-prod-mysql-ro.internal.leapstack.cn')

### 查询
主要封装了关于查询的便利方法

In [10]:
sqltemplate = """
        show databases
        """

In [11]:
mysql.queryAndFetchAll(sqltemplate, cols=['db_name'])

,db_name
0,information_schema
1,battleaxe_prod
2,bc_be_bxy_prod
3,bc_be_ddh_prod
4,bc_be_hic_prod
5,bc_be_prod
6,bc_be_prod_new
7,bc_be_prod_nn
8,cheetah_prod
9,claim_registration_center


In [31]:
sqltemplate = """
        SELECT insured_person_id_no, claim_id from claim_apply_record
        """

dfr1 = mysql.queryAndFetchAll(sqltemplate, cols=['insured_person_id_no', 'claim_id'])

In [32]:
dfr1 = dfr1.dropna(how='any')

In [33]:
dfr1.shape

(50526, 2)

In [39]:
dfr1.claim_id.nunique()

50526

In [44]:
invoice_data_dict = {
    'invoice_data_id': 'invoice_data_id',
    'claim_id': 'claim_id',
    'bill_sno': '收据号',
    'bill_type': '收据类型',
    'bill_amt': '收据总金额',
    'bill_date': '发票日期',
    'cash_pay_amt': '现金支付金额',
    'account_pay_amt': '医保账户支付金额',
    'overall_pay_amt': '统筹支付金额',
    'attach_pay_amt': '附加支付金额',
    'self_pay_amt': '自费金额金额',
    'classification_pay_amt': '分类自负金额',
    'third_pay_amt': '第三方支付金额',
    'care_date': '就诊日期',
    'accident_subtype': '出险类型',
    'hospital_name': '就诊医院',
    'ill_code': '疾病主要诊断',
    'sub_ill_code': '疾病次要诊断',
    'in_date': '入院日期',
    'out_date': '出院日期',
    'in_days': '住院天数',
    'hospital_grade': '医院等级',
    'ill_code_zh': '疾病名称中文',
    'hospital_level': '医院等级',
    'hospital_code': '医院代码',
}

In [50]:
sqltemplate = f"SELECT {','.join(invoice_data_dict.keys())} from invoice_data"

dfr2 = mysql2.queryAndFetchAll(sqltemplate, cols=invoice_data_dict.keys())

In [46]:
dfr2.shape

(239540, 25)

In [42]:
import pandas as pd

In [51]:
dfr = pd.merge(dfr1, dfr2)

In [52]:
dfr.shape

(142431, 26)

In [54]:
item_data_dict = {
    'item_data_id': 'item_data_id',
    'item_name': '账单名称',
    'medical_pay_amt': '医保给付金额',
    'self_pay_amt': '自费金额',
    'third_pay_amt': '分类自负',
    'item_sub_name': '费用名称',
    'item_pay': '发生金额',
    'self_pay_rate': '自付比例',
    'classification_pay_amt': '分类自负金额',
    'item_unit_pay': '单价',
    'item_cnt': '数量',
    'invoice_data_id': 'invoice_data_id',
    'medical_pay_type': 'medical_pay_type',
}

In [ ]:
# 商保字段和cy表字段的对应关系
{
    'medical_pay_type': 'applyPayLevel',   # 需处理，'甲类'->'甲'
    'item_name': 'chargeType', 
    'item_unit_pay': 'price', 
    'item_cnt': 'quantity', 
    '': 'money',  #  item_unit_pay * item_cnt
    'self_pay_amt': 'selfAmount', 
    '': '', 
    '': '', 
    'hospital_grade': 'hospLevel', 
    'hospital_code': 'hospNo', 
    'in_days': 'hospDays', 
    'item_sub_name': 'hospProjectName', 
    'ill_code_zh': 'outHospDiagnosisName', 
    'ill_code': 'outHospDiagnosisCode', 
    'in_date': 'inHospDate', 
    'bill_amt': 'billSum', 
    'self_pay_amt': 'billSelf', 
    'overall_pay_amt': 'rs_overallPay', 
    'account_pay_amt': 'accountPay', 
    '': '', 
    '': '', 
    '': '', 
}

In [ ]:
sqltemplate = f"SELECT {','.join(item_data_dict.keys())} from item_data"

dfr2 = mysql2.queryAndFetchAll(sqltemplate, cols=item_data_dict.keys())

### 关于除查询外的其它操作
就是对原生pymysql的简单封装